In [1]:
"""
Chromosome-wise Evaluation for HiC-SuperNet
============================================

Evaluates model performance on specific chromosomes:
- Chr4, Chr14, Chr16, Chr20
- Computes Mean ± SD across chromosomes
- Comparable to DiCARN-DNase paper format
"""

import tensorflow as tf
from tensorflow import keras
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import pandas as pd

# ==================== GPU Configuration ====================
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(f"⚠ GPU warning: {e}")

# ==================== Load Custom Layers ====================
class MultiScaleDilatedResBlock(keras.layers.Layer):
    def __init__(self, filters, **kwargs):
        super(MultiScaleDilatedResBlock, self).__init__(**kwargs)
        self.filters = filters
        
        self.conv_d1 = keras.layers.Conv2D(filters // 3, 3, padding='same', dilation_rate=1)
        self.conv_d2 = keras.layers.Conv2D(filters // 3, 3, padding='same', dilation_rate=2)
        self.conv_d4 = keras.layers.Conv2D(filters // 3, 3, padding='same', dilation_rate=4)
        
        self.bn1 = keras.layers.BatchNormalization()
        self.relu1 = keras.layers.ReLU()
        
        self.conv2 = keras.layers.Conv2D(filters, 3, padding='same')
        self.bn2 = keras.layers.BatchNormalization()
        
        self.shortcut_conv = keras.layers.Conv2D(filters, 1, padding='same')
        
    def call(self, x, training=False):
        shortcut = self.shortcut_conv(x)
        
        d1 = self.conv_d1(x)
        d2 = self.conv_d2(x)
        d4 = self.conv_d4(x)
        
        out = keras.layers.Concatenate()([d1, d2, d4])
        out = self.bn1(out, training=training)
        out = self.relu1(out)
        
        out = self.conv2(out)
        out = self.bn2(out, training=training)
        
        out = keras.layers.Add()([out, shortcut])
        out = keras.layers.ReLU()(out)
        
        return out
    
    def get_config(self):
        config = super().get_config()
        config.update({'filters': self.filters})
        return config

class DualAttention(keras.layers.Layer):
    def __init__(self, filters, reduction=8, **kwargs):
        super(DualAttention, self).__init__(**kwargs)
        self.filters = filters
        self.reduction = reduction
        
        self.gap = keras.layers.GlobalAveragePooling2D(keepdims=True)
        self.gmp = keras.layers.GlobalMaxPooling2D(keepdims=True)
        self.fc1 = keras.layers.Dense(filters // reduction, activation='relu')
        self.fc2 = keras.layers.Dense(filters)
        
        self.spatial_conv = keras.layers.Conv2D(1, 7, padding='same')
        
    def call(self, x):
        avg_pool = self.gap(x)
        max_pool = self.gmp(x)
        
        avg_pool = tf.reshape(avg_pool, [-1, self.filters])
        max_pool = tf.reshape(max_pool, [-1, self.filters])
        
        avg_out = self.fc2(self.fc1(avg_pool))
        max_out = self.fc2(self.fc1(max_pool))
        
        channel_att = tf.nn.sigmoid(avg_out + max_out)
        channel_att = tf.reshape(channel_att, [-1, 1, 1, self.filters])
        
        x = x * channel_att
        
        avg_out = tf.reduce_mean(x, axis=-1, keepdims=True)
        max_out = tf.reduce_max(x, axis=-1, keepdims=True)
        concat = keras.layers.Concatenate()([avg_out, max_out])
        
        spatial_att = tf.nn.sigmoid(self.spatial_conv(concat))
        x = x * spatial_att
        
        return x
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'filters': self.filters,
            'reduction': self.reduction
        })
        return config

def improved_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1, tf.shape(y_true)[1], tf.shape(y_true)[2], 1])
    y_pred = tf.reshape(y_pred, [-1, tf.shape(y_pred)[1], tf.shape(y_pred)[2], 1])
    
    mse_loss = tf.reduce_mean(tf.square(y_true - y_pred))
    mae_loss = tf.reduce_mean(tf.abs(y_true - y_pred))
    
    y_true_flat = tf.reshape(y_true, [tf.shape(y_true)[0], -1])
    y_pred_flat = tf.reshape(y_pred, [tf.shape(y_pred)[0], -1])
    
    y_true_mean = tf.reduce_mean(y_true_flat, axis=1, keepdims=True)
    y_pred_mean = tf.reduce_mean(y_pred_flat, axis=1, keepdims=True)
    
    y_true_centered = y_true_flat - y_true_mean
    y_pred_centered = y_pred_flat - y_pred_mean
    
    numerator = tf.reduce_sum(y_true_centered * y_pred_centered, axis=1)
    denominator = tf.sqrt(
        tf.reduce_sum(tf.square(y_true_centered), axis=1) * 
        tf.reduce_sum(tf.square(y_pred_centered), axis=1)
    )
    
    pearson = numerator / (denominator + 1e-8)
    pearson_loss = 1 - tf.reduce_mean(pearson)
    
    ssim_loss = 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))
    
    total_loss = (
        0.4 * mse_loss + 
        0.2 * mae_loss +
        0.3 * pearson_loss + 
        0.1 * ssim_loss
    )
    
    return total_loss

# ==================== Metrics Functions ====================
def calculate_ssim(img1, img2, C1=1e-4, C2=9e-4):
    """Calculate SSIM between two images"""
    img1 = img1.flatten()
    img2 = img2.flatten()
    
    mu1 = np.mean(img1)
    mu2 = np.mean(img2)
    sigma1 = np.std(img1)
    sigma2 = np.std(img2)
    sigma12 = np.mean((img1 - mu1) * (img2 - mu2))
    
    ssim = ((2 * mu1 * mu2 + C1) * (2 * sigma12 + C2)) / \
           ((mu1**2 + mu2**2 + C1) * (sigma1**2 + sigma2**2 + C2))
    
    return ssim

def calculate_psnr(img1, img2, max_val=None):
    """Calculate PSNR between two images"""
    mse = mean_squared_error(img1.flatten(), img2.flatten())
    if mse == 0:
        return float('inf')
    
    if max_val is None:
        max_val = max(np.max(img1), np.max(img2))
    
    psnr = 20 * math.log10(max_val / math.sqrt(mse))
    return psnr

def calculate_metrics(pred, target):
    """Calculate all metrics for a single sample"""
    pred_flat = pred.flatten()
    target_flat = target.flatten()
    
    mse = mean_squared_error(target_flat, pred_flat)
    mae = mean_absolute_error(target_flat, pred_flat)
    
    if np.std(pred_flat) == 0 or np.std(target_flat) == 0:
        pcc = np.nan
    else:
        pcc, _ = pearsonr(pred_flat, target_flat)
    
    ssim = calculate_ssim(pred, target)
    psnr = calculate_psnr(pred, target)
    
    return {
        'mse': mse,
        'mae': mae,
        'pcc': pcc,
        'ssim': ssim,
        'psnr': psnr
    }

# ==================== Chromosome-wise Evaluation ====================
def evaluate_chromosome_wise(model, test_npz_path, target_chromosomes=['chr4', 'chr14', 'chr16', 'chr20'], 
                             batch_size=16):
    """
    Evaluate model performance on specific chromosomes
    
    Args:
        model: Trained Keras model
        test_npz_path: Path to test npz file
        target_chromosomes: List of chromosome names to evaluate
        batch_size: Batch size for prediction
    
    Returns:
        DataFrame with chromosome-wise results
    """
    
    print("="*80)
    print("📊 CHROMOSOME-WISE EVALUATION")
    print("="*80)
    
    # Load test data
    print(f"\n📂 Loading test data from: {test_npz_path}")
    data = np.load(test_npz_path, allow_pickle=True)
    lr_data = data['data']
    hr_data = data['target']
    inds = data['inds']
    
    print(f"✓ Total samples loaded: {len(lr_data)}")
    
    # Process chromosome information
    # inds format: typically contains chromosome and position info
    # Extract chromosome labels from inds
    chromosome_labels = []
    for ind in inds:
        # ind is typically in format: (chr_num, start_pos, end_pos) or similar
        if isinstance(ind, (list, tuple, np.ndarray)):
            chr_num = ind[0] if len(ind) > 0 else None
        else:
            chr_num = ind
        
        # Convert to chromosome name
        if chr_num is not None:
            if isinstance(chr_num, (int, np.integer)):
                chr_name = f'chr{chr_num}'
            else:
                chr_name = str(chr_num).lower()
        else:
            chr_name = 'unknown'
        
        chromosome_labels.append(chr_name)
    
    # Group samples by chromosome
    chromosome_dict = {}
    for i, chr_name in enumerate(chromosome_labels):
        if chr_name not in chromosome_dict:
            chromosome_dict[chr_name] = []
        chromosome_dict[chr_name].append(i)
    
    print(f"\n📋 Chromosomes found in dataset:")
    for chr_name, indices in sorted(chromosome_dict.items()):
        print(f"   {chr_name}: {len(indices)} samples")
    
    # Evaluate each target chromosome
    results = {}
    
    for chr_name in target_chromosomes:
        chr_name_lower = chr_name.lower()
        
        # Find matching chromosome
        matching_chr = None
        for key in chromosome_dict.keys():
            if chr_name_lower in key.lower():
                matching_chr = key
                break
        
        if matching_chr is None:
            print(f"\n⚠ Warning: {chr_name} not found in dataset, skipping...")
            continue
        
        print(f"\n{'='*80}")
        print(f"🧬 Evaluating {chr_name.upper()}...")
        print(f"{'='*80}")
        
        indices = chromosome_dict[matching_chr]
        print(f"   Samples: {len(indices)}")
        
        # Extract chromosome-specific data
        chr_lr = lr_data[indices]
        chr_hr = hr_data[indices]
        
        # Fix dimensions if needed
        if chr_lr.ndim == 4 and chr_lr.shape[1] == 1:
            chr_lr = chr_lr[:, 0, :, :]
        if chr_hr.ndim == 4 and chr_hr.shape[1] == 1:
            chr_hr = chr_hr[:, 0, :, :]
        
        if chr_lr.ndim == 3:
            chr_lr = np.expand_dims(chr_lr, axis=-1)
        if chr_hr.ndim == 3:
            chr_hr = np.expand_dims(chr_hr, axis=-1)
        
        # Predict in batches
        chr_metrics = []
        num_samples = len(chr_lr)
        
        for i in range(0, num_samples, batch_size):
            end_idx = min(i + batch_size, num_samples)
            batch_lr = chr_lr[i:end_idx].astype('float32')
            batch_hr = chr_hr[i:end_idx].astype('float32')
            
            # Predict
            batch_pred = model.predict(batch_lr, verbose=0)
            
            # Calculate metrics for each sample in batch
            for j in range(len(batch_lr)):
                metrics = calculate_metrics(
                    batch_pred[j, :, :, 0],
                    batch_hr[j, :, :, 0]
                )
                chr_metrics.append(metrics)
            
            if (i + batch_size) % 100 == 0 or end_idx == num_samples:
                print(f"   Processed: {end_idx}/{num_samples} samples...")
        
        # Average metrics for this chromosome
        avg_metrics = {}
        for key in chr_metrics[0].keys():
            values = [m[key] for m in chr_metrics if not np.isnan(m[key])]
            avg_metrics[key] = np.mean(values) if values else np.nan
        
        results[chr_name.upper()] = avg_metrics
        
        print(f"\n   📈 Results for {chr_name.upper()}:")
        print(f"      MSE:  {avg_metrics['mse']:.6f}")
        print(f"      MAE:  {avg_metrics['mae']:.6f}")
        print(f"      PCC:  {avg_metrics['pcc']:.4f}")
        print(f"      SSIM: {avg_metrics['ssim']:.4f}")
        print(f"      PSNR: {avg_metrics['psnr']:.2f} dB")
    
    return results

def format_results_table(results):
    """
    Format results in DiCARN-DNase paper style
    
    Returns:
        Formatted string table
    """
    
    if not results:
        return "No results to display"
    
    # Create DataFrame
    metrics = ['SSIM', 'PSNR', 'MSE', 'MAE', 'PCC']
    chromosomes = sorted(results.keys())
    
    data = {}
    for metric in metrics:
        metric_lower = metric.lower()
        values = []
        for chr_name in chromosomes:
            if metric_lower in results[chr_name]:
                values.append(results[chr_name][metric_lower])
            else:
                values.append(np.nan)
        
        # Calculate mean and std
        valid_values = [v for v in values if not np.isnan(v)]
        mean_val = np.mean(valid_values) if valid_values else np.nan
        std_val = np.std(valid_values) if valid_values else np.nan
        
        # Add to data
        data[metric] = values + [mean_val, std_val]
    
    # Create DataFrame
    index = chromosomes + ['Mean', 'Std']
    df = pd.DataFrame(data, index=index)
    
    return df

def print_comparison_table(results, method_name='HiC-SuperNet'):
    """
    Print results in a format comparable to DiCARN-DNase paper Table 1
    """
    
    print("\n" + "="*80)
    print(f"📊 RESULTS TABLE - {method_name}")
    print("="*80)
    
    chromosomes = sorted(results.keys())
    
    # Prepare data
    metrics_data = {
        'SSIM': [],
        'PSNR': [],
        'MSE': [],
        'MAE': [],
        'PCC': []
    }
    
    for chr_name in chromosomes:
        for metric in metrics_data.keys():
            metrics_data[metric].append(results[chr_name][metric.lower()])
    
    # Calculate mean ± std
    print(f"\n{'Metric':<10}", end="")
    for chr_name in chromosomes:
        print(f"{chr_name:<12}", end="")
    print(f"{'Mean ± SD':<20}")
    print("-" * 80)
    
    for metric, values in metrics_data.items():
        mean_val = np.mean(values)
        std_val = np.std(values)
        
        print(f"{metric:<10}", end="")
        for val in values:
            if metric == 'PSNR':
                print(f"{val:<12.2f}", end="")
            else:
                print(f"{val:<12.6f}" if metric in ['MSE', 'MAE'] else f"{val:<12.4f}", end="")
        
        if metric == 'PSNR':
            print(f"{mean_val:.2f} ± {std_val:.2f}")
        elif metric in ['MSE', 'MAE']:
            print(f"{mean_val:.6f} ± {std_val:.6f}")
        else:
            print(f"{mean_val:.4f} ± {std_val:.4f}")
    
    print("="*80)
    
    # Print in bold the best scores section
    print("\n🏆 SUMMARY (Mean ± SD):")
    print("-" * 40)
    for metric, values in metrics_data.items():
        mean_val = np.mean(values)
        std_val = np.std(values)
        
        if metric == 'PSNR':
            print(f"   {metric}: {mean_val:.2f} ± {std_val:.2f} dB")
        elif metric in ['MSE', 'MAE']:
            print(f"   {metric}: {mean_val:.6f} ± {std_val:.6f}")
        else:
            print(f"   {metric}: {mean_val:.4f} ± {std_val:.4f}")
    print("="*80)

# ==================== Main Evaluation Function ====================
def main():
    """
    Main evaluation script
    """
    
    # Configuration
    MODEL_PATH = 'best_hic_supernet.keras'
    TEST_NPZ_PATH = 'hicarn_10kb40kb_c40_s40_b201_nonpool_HMEC_test.npz'
    TARGET_CHROMOSOMES = ['chr4', 'chr14', 'chr16', 'chr20']
    BATCH_SIZE = 16
    
    print("\n" + "="*80)
    print("🧬 CHROMOSOME-WISE EVALUATION FOR HiC-SuperNet")
    print("="*80)
    
    # Load model
    print(f"\n📂 Loading model from: {MODEL_PATH}")
    model = keras.models.load_model(
        MODEL_PATH,
        custom_objects={
            'improved_loss': improved_loss,
            'MultiScaleDilatedResBlock': MultiScaleDilatedResBlock,
            'DualAttention': DualAttention
        }
    )
    print("✓ Model loaded successfully")
    
    # Evaluate chromosome-wise
    results = evaluate_chromosome_wise(
        model=model,
        test_npz_path=TEST_NPZ_PATH,
        target_chromosomes=TARGET_CHROMOSOMES,
        batch_size=BATCH_SIZE
    )
    
    # Print results table
    print_comparison_table(results, method_name='HiC-SuperNet')
    
    # Save results to CSV
    df = format_results_table(results)
    csv_filename = 'chromosome_wise_results.csv'
    df.to_csv(csv_filename)
    print(f"\n💾 Results saved to: {csv_filename}")
    
    print("\n✅ Evaluation complete!")
    
    return results

if __name__ == "__main__":
    results = main()

C:\Users\Proly Kumar\AppData\Local\anaconda3\envs\tensorflow-gpu-01\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(



🧬 CHROMOSOME-WISE EVALUATION FOR HiC-SuperNet

📂 Loading model from: best_hic_supernet.keras
✓ Model loaded successfully
📊 CHROMOSOME-WISE EVALUATION

📂 Loading test data from: hicarn_10kb40kb_c40_s40_b201_nonpool_HMEC_test.npz
✓ Total samples loaded: 11144

📋 Chromosomes found in dataset:
   chr14: 2368 samples
   chr16: 2060 samples
   chr20: 1598 samples
   chr4: 5118 samples

🧬 Evaluating CHR4...
   Samples: 5118
   Processed: 400/5118 samples...
   Processed: 800/5118 samples...
   Processed: 1200/5118 samples...
   Processed: 1600/5118 samples...
   Processed: 2000/5118 samples...
   Processed: 2400/5118 samples...
   Processed: 2800/5118 samples...
   Processed: 3200/5118 samples...
   Processed: 3600/5118 samples...
   Processed: 4000/5118 samples...
   Processed: 4400/5118 samples...
   Processed: 4800/5118 samples...
   Processed: 5118/5118 samples...

   📈 Results for CHR4:
      MSE:  0.000738
      MAE:  0.005218
      PCC:  0.4219
      SSIM: 0.9200
      PSNR: 18.86 dB
